### Manipulação de Dados com DataFrames no PySpark - Operação READ

**Objetivo:** Nesta seção, você aprenderá como carregar dados utilizando DataFrames no PySpark. Os DataFrames são uma das abstrações mais poderosas do Apache Spark, fornecendo uma API de alto nível para o processamento eficiente de dados distribuídos.

#### Criação de DataFrames

##### A partir de Arquivos (CSV, JSON, Parquet, etc.)

O PySpark suporta a leitura de múltiplos formatos de arquivos, como CSV, JSON, Parquet, ORC, Avro, entre outros. A seguir, exploramos as opções disponíveis para cada formato, com exemplos detalhados para uma compreensão completa.

---

### Carregando Dados de Arquivos CSV

Documentação de referência: [CSV Data Source](https://spark.apache.org/docs/latest/sql-data-sources-csv.html)

O Spark oferece uma variedade de opções de parametrização ao ler arquivos CSV, facilitando o tratamento de arquivos com diferentes formatos e estruturas.

**Opções Comuns de Parametrização para CSV:**
- **`header`**: Especifica se o arquivo CSV tem um cabeçalho (`True` ou `False`).
- **`sep`**: Define o delimitador de colunas (ex: `','`, `';'`).
- **`inferSchema`**: Infere automaticamente o tipo de dado das colunas (`True` ou `False`).
- **`nullValue`**: Define valores específicos no CSV que devem ser interpretados como `null`.
- **`quote`**: Define o caractere usado para aspas (ex: `'"'`).
- **`escape`**: Define o caractere usado para escapar aspas (ex: `'\\'`).
- **`encoding`**: Especifica a codificação do arquivo (ex: `'UTF-8'`).

**Exemplo Prático:**


In [0]:
%sql
create volume if not exists `laboratorio-spark`;

In [0]:
%sh
wget -O /Volumes/workspace/default/laboratorio-spark/airtravel.csv https://raw.githubusercontent.com/rafael-negrao/laboratorio-spark/main/dados/airtravel.csv

In [0]:
file = "/Volumes/workspace/default/laboratorio-spark/airtravel.csv"

# Lendo o CSV com opções de parametrização
df_csv = spark.read.csv(
    file, 
    header=True,         # Indica que o arquivo tem cabeçalho
    sep=',',             # Delimitador das colunas
    inferSchema=True,    # Infere automaticamente os tipos das colunas
    nullValue='NA',      # Define valores nulos
    quote='"',           # Define o caractere de aspas
    escape='\\',         # Caractere de escape
    encoding='UTF-8'     # Codificação do arquivo
)
display(df_csv)


---

### Carregando Dados de Arquivos JSON

Documentação de referência: [JSON Data Source](https://spark.apache.org/docs/latest/sql-data-sources-json.html)

O Spark suporta dois formatos principais de JSON: **inline** (todas as entradas em uma única linha) e **multilinha** (entradas separadas por linhas).

#### JSON Inline

**Exemplo de Arquivo JSON Inline:**
```json
{"name": "Alice", "age": 30}
{"name": "Bob", "age": 25}
```

**Opções Comuns de Parametrização para JSON Inline:**
- **`multiline`**: Deve ser `False` (padrão) para JSON inline.
- **`primitivesAsString`**: Interpreta tipos primitivos como strings (`True` ou `False`).
- **`allowSingleQuotes`**: Permite o uso de aspas simples para delimitar strings (`True` ou `False`).
- **`mode`**: Define o comportamento em caso de erros (`PERMISSIVE`, `DROPMALFORMED`, `FAILFAST`).

**Exemplo Prático:**


In [0]:
dbutils.fs.help("put")

In [0]:
file = "/Volumes/workspace/default/laboratorio-spark/exemplo.json"

dbutils.fs.put(
    file, 
    """{"name": "Alice", "age": 30}
{"name": "Bob", "age": 25}""",
    overwrite=True)

# Lendo JSON Inline
df_json_inline = spark.read.json(
    file, 
    multiLine=False,            # JSON inline
    primitivesAsString=True,    # Converte primitivos para string
    allowSingleQuotes=True,     # Permite aspas simples
    mode='PERMISSIVE'           # Modo permissivo para erros
)
display(df_json_inline)


#### JSON Multilinha

**Exemplo de JSON Multilinha:**
```json
[
  {"name": "Alice", "age": 30},
  {"name": "Bob", "age": 25}
]
```

**Opções Comuns de Parametrização para JSON Multilinha:**
- **`multiline`**: Deve ser `True` para JSON multilinha.
- **`primitivesAsString`**, **`allowSingleQuotes`**, **`mode`**: As mesmas opções que o JSON inline.

**Exemplo Prático:**


In [0]:
file = "/Volumes/workspace/default/laboratorio-spark/exemplo-multilinha.json"

dbutils.fs.put(
    file, 
    """[
  {"name": "Alice", "age": 30},
  {"name": "Bob", "age": 25}
]""",
    overwrite=True)


# Lendo JSON Multilinha
df_json_multiline = spark.read.json(
    file, 
    multiLine=True,             # JSON Multilinha
    primitivesAsString=True,    # Converte primitivos para string
    allowSingleQuotes=True,     # Permite aspas simples
    mode='PERMISSIVE'           # Modo permissivo para erros
)
display(df_json_multiline)


---

### Carregando Dados de Arquivos Parquet

Documentação de referência: [Parquet Data Source](https://spark.apache.org/docs/latest/sql-data-sources-parquet.html)

O formato Parquet é otimizado para leitura rápida e eficiente, especialmente para grandes volumes de dados.

**Opções Comuns de Parametrização para Parquet:**
- **`mergeSchema`**: Combina esquemas de múltiplos arquivos (`True` ou `False`).
- **`spark.sql.parquet.filterPushdown`**: Controla o pushdown de filtros para otimizar consultas (habilitado por padrão).
- **`spark.sql.parquet.enableVectorizedReader`**: Habilita a leitura vetorizada para melhorar a performance (habilitado por padrão).

**Exemplo Prático:**


In [0]:
diretorio = "/Volumes/workspace/default/laboratorio-spark/example_parquet"

# Criando um DataFrame de exemplo e salvando como Parquet
data = [("Alice", 30), ("Bob", 25)]
df = spark.createDataFrame(data, ["name", "age"])
df.write.mode("overwrite").parquet(diretorio)

# Lendo o arquivo Parquet com opções
df_parquet = (
    spark
        .read
        .option("mergeSchema", "true") # Combina esquemas se múltiplos arquivos estiverem presentes
        .parquet(diretorio)
) 

display(df_parquet)

In [0]:
# Lendo o arquivo Parquet com opções
df_parquet2 = (
    spark
        .read
        .format("parquet")
        .option("mergeSchema", "true") # Combina esquemas se múltiplos arquivos estiverem presentes
        .load(diretorio)
) 
display(df_parquet2)


---

### Carregando Dados de Arquivos Text

Documentação de referência: [Text Data Source](https://spark.apache.org/docs/latest/sql-data-sources-text.html)

O formato de texto é simples e pode ser útil para dados não estruturados ou entradas de logs.

**Opções Comuns de Parametrização para Text:**
- **`wholetext`**: Lê o arquivo inteiro como uma única entrada (`True` ou `False`).
- **`lineSep`**: Define o separador de linha (ex: `'\n'`).

**Exemplo Prático:**


In [0]:

file = "/Volumes/workspace/default/laboratorio-spark/exemplo-olamundo.txt"

dbutils.fs.put(
    file, 
    """Hello World\nWelcome to PySpark""",
    overwrite=True)

# Lendo o arquivo de texto com opções
df_text = spark.read.text(
    file, 
    wholetext=False,     # Lê o arquivo linha por linha
    lineSep='\n',        # Define o separador de linha
)
display(df_text)

`[INFO]: FIM DO NOTEBOOK`